In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import orbax
from flax.training import orbax_utils

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *

In [2]:
world = SDFWorld()
world.show_in_jupyter()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7,8], [0.04, 0.04])

In [4]:
obj_start = Mesh(world.vis, "obj_start", "./sdf_world/assets/object/mesh.obj",
                 alpha=0.5)
d, w, h = obj_start.mesh.bounding_box.primitive.extents
obj_start.set_translate([0.4, -0.3, h/2])

obj_goal = Mesh(world.vis, "obj_goal", "./sdf_world/assets/object/mesh.obj",
                alpha=0.5)
goal_pose = SE3.from_rotation_and_translation(
    rotation=SO3.from_x_radians(jnp.pi/2),
    translation=jnp.array([0.4, 0.3, w/2])
)
obj_goal.set_pose(goal_pose)


hand_model = RobotModel(HAND_URDF, PANDA_PACKAGE, True)
hand1 = Robot(world.vis, "hand1", hand_model, color="yellow", alpha=0.5)
hand2 = Robot(world.vis, "hand2", hand_model, color="yellow", alpha=0.5)
qhand = jnp.array([0.04, 0.04])
hand1.set_joint_angles(qhand)
hand2.set_joint_angles(qhand)

ground = Box(world.vis, "ground", [2, 2, 0.5], "gray", alpha=0.5)
ground.set_translate([0,0,-ground.lengths[-1]/2])

import pickle
with open("./sdf_world/assets/object"+'/info.pkl', 'rb') as f:
    obj_data = pickle.load(f)
scale_to_norm = obj_data["scale_to_norm"]

In [23]:
obj_points = obj_start.mesh.sample(200)
obj_points = farthest_point_sampling(obj_points, 20)

In [5]:
# visualize
def set_hand_pose(pose, hand:Robot):
    ee_base_pose = pose @ SE3.from_translation(jnp.array([0,0,-0.105]))
    hand.set_pose(ee_base_pose)

In [6]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()

grasp_net_dict = orbax_checkpointer.restore("model/grasp_net")
manip_net_dict = orbax_checkpointer.restore("model/manip_net")

grasp_net = GraspNet(10)
manip_net = ManipNet(64)
grasp_fn = lambda x: grasp_net.apply(grasp_net_dict['params'], x)
manip_fn = lambda posevecs: manip_net.apply(manip_net_dict["params"], posevecs)[0]

In [7]:
def grasp_reconst(g:Array):
    rot = SO3(grasp_fn(g)[1:5]).normalize()
    trans = g/scale_to_norm
    return SE3.from_rotation_and_translation(rot, trans)

In [8]:
g = jnp.array([0,0,0.])
grasp_pose = grasp_reconst(g)
set_hand_pose(obj_start.pose @ grasp_pose, hand1)
set_hand_pose(obj_goal.pose @ grasp_pose, hand2)

In [209]:
# differential modules:
# grasp prob
tcp_wrt_ee = SE3.from_translation(jnp.array([0,0,-0.105]))
hand_pc = hand1.get_surface_points_fn(qhand)
hand_pc = jax.vmap(tcp_wrt_ee.apply)(hand_pc)
env = SDFContainer([ground], 0.05)

def get_grasp_logit(g:Array):
    return grasp_fn(g)[0]

def SE3_interp(pose1:SE3, pose2:SE3, num=5):
    diff_vec_pos = pose2.translation() - pose1.translation()
    diff_vec_rot = (pose1.rotation().inverse() @ pose2.rotation()).log()
    result = []
    for r in range(num-1):
        ratio = (r)/(num-1)
        pos = pose1.translation() + diff_vec_pos* ratio
        rot = pose1.rotation() @ SO3.exp(diff_vec_rot*ratio)
        result += [SE3.from_rotation_and_translation(rot, pos)]
    result += [pose2]
    return result

def grasp_to_posevecs(grasp_pose, apply_zflip=False):
    if apply_zflip:
        zflip = SE3.from_rotation(SO3.from_z_radians(jnp.pi))
    else:
        zflip = SE3.identity()
    poses = []
    pose_pick = obj_start.pose @ grasp_pose @ zflip
    pose_place = obj_goal.pose @ grasp_pose @ zflip
    poses = [pose_pick, pose_place]
    #SE3_interp(pose_pick, pose_place)
    poses = jnp.vstack([pose.parameters() for pose in poses])
    return poses

# Penetration
assign_points_fn = lambda posevec, points :jax.vmap(SE3(posevec).apply)(points)
assign_hands_points_fn = lambda posevecs: jnp.vstack(
    jax.vmap(assign_points_fn, in_axes=(0, None))(posevecs, hand_pc))
# def assign_obj_points_fn(posevec, grasp_pose):
#     obj_pose = SE3(posevec) @ grasp_pose.inverse()
#     return jax.vmap(obj_pose.apply)(obj_points)
# def assign_ee_and_obj_points(posevecs, grasp_pose):
#     ee_points = assign_hands_points_fn(posevecs)
#     obj_points = jnp.vstack(jax.vmap(assign_obj_points_fn, in_axes=(0, None))(posevecs, grasp_pose))
#     return jnp.vstack([ee_points, obj_points])

def penetration(posevecs):
    points = assign_hands_points_fn(posevecs)
    return env.penetration_sum(points)

# Manipulability
ws_r = 1.
ws_center = jnp.array([0,0,0.5])
def get_manip_value(posevec):
    _, xyz = posevec[:4], posevec[4:]
    is_out_of_bound = jnp.linalg.norm(xyz - ws_center) > ws_r
    return jax.lax.cond(
        is_out_of_bound, lambda x:jnp.array(0.), manip_fn, posevec)
def manipulability(posevecs):
    return jax.vmap(get_manip_value)(posevecs)

def constraints(g, apply_zflip=False):
    grasp_pose = grasp_reconst(g)
    posevecs = grasp_to_posevecs(grasp_pose, apply_zflip)
    logit = get_grasp_logit(g)
    penet = penetration(posevecs)
    manips = manipulability(posevecs)
    return jnp.hstack([logit, penet, manips])

def soft_constraint(g):
    grasp_pose = grasp_reconst(g)
    posevecs = grasp_to_posevecs(grasp_pose)
    logit = get_grasp_logit(g)
    penet = penetration(posevecs)
    manips = manipulability(posevecs)
    return - logit + penet*5 - manips.min()

In [63]:
global pc
pc = None
def view_hands(g):
    global pc
    grasp_pose = grasp_reconst(g)
    posevecs = grasp_to_posevecs(grasp_pose)
    set_hand_pose(SE3(posevecs[0]), hand1)
    set_hand_pose(SE3(posevecs[1]), hand2)
    points = assign_hands_points_fn(posevecs)
    if pc is None:
        pc = PointCloud(world.vis, "pc", points, 0.01, "red")
    else:
        pc.reload(points=points)

In [210]:
import optax

optimizer = optax.adam(0.05)
state = optimizer.init(g)
eval = jax.jit(jax.value_and_grad(soft_constraint)).lower(g).compile()

In [1400]:
g = np.random.normal(size=3)
g = g/jnp.linalg.norm(g)
view_hands(g)

In [1401]:
g_init = g.copy()

In [1468]:
g = g_init

In [1530]:
val, grads = eval(g)
updates, state = optimizer.update(grads, state)
g = optax.apply_updates(g, updates)
print(f"value: {val}")
view_hands(g)

# grasp_pose = grasp_reconst(g)
# posevecs = grasp_to_posevecs(
# grasp_pose)
# logit = get_grasp_logit(g)
# penet = penetration(posevecs)
# manips = manipulability(posevecs)
# print(logit, penet)

value: -3.809521198272705


In [2255]:
from sdf_world.nlp import NLP
prob = NLP()

In [2266]:
from functools import partial
prob.add_var("g", 3, -1*np.ones(3), np.ones(3))
prob.add_con("con", 4, ["g"], partial(constraints, apply_zflip=True), 
             lower=np.array([0.5, 0., 0.1, 0.1]),
             upper=np.array([jnp.inf, 0., jnp.inf, jnp.inf])
)
ee_pose_init = SE3(panda_model.fk_fn(panda.neutral)[-1])
def SE3_distance(pose1, pose2):
    pos_err = pose1.translation() - pose2.translation()
    rot_err = (pose1.rotation().inverse()@pose2.rotation()).log()
    return safe_2norm(pos_err) + safe_2norm(rot_err)/3

def objective(g):
    grasp_pose = grasp_reconst(g)
    posevecs = grasp_to_posevecs(grasp_pose, False)
    poses = [ee_pose_init, SE3(posevecs[0]), SE3(posevecs[1]), ee_pose_init]
    error = 0.
    for i in range(len(poses)-1):
        pose1, pose2 = poses[i], poses[i+1]
        error += SE3_distance(pose1, pose2)
    return error

prob.add_objective(objective)
prob.build()

In [2247]:
panda.set_joint_angles(panda.neutral)

In [2267]:
xsol, info = prob.solve(g, viol_tol=0.01)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.8153131e+00 2.30e-03 1.02e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [2270]:
view_hands(xsol)

In [2271]:
constraints(xsol)

Array([2.4893446 , 0.        , 0.2854541 , 0.38682485], dtype=float32)

In [2272]:
objective(xsol)

Array(3.4786894, dtype=float32)

In [2283]:
from scipy.optimize import minimize
from functools import partial
@jax.jit
def _pose_error(target_pose, q):
    q = jnp.hstack([q, 0.04, 0.04])
    ee_pose = SE3(panda_model.fk_fn(q)[-1])
    pos_error = ee_pose.translation() - target_pose.translation()
    pos_error = jnp.sum(pos_error **2)
    orn_error = (target_pose.inverse()@ee_pose).rotation().log()
    orn_error = jnp.sum(orn_error ** 2)
    return pos_error + orn_error/3

_pose_error_vg = jax.jit(jax.value_and_grad(_pose_error, argnums=1)).lower(SE3.identity(), jnp.zeros(7)).compile()

def inverse_kinematics(pose, x0=panda_model.neutral[:7]):
    pose_error = partial(_pose_error_vg, pose)
    res = minimize(
        pose_error, x0, 
        method="SLSQP", jac=True,
        bounds=[(lb, ub) for lb, ub in zip(panda.lb, panda.ub)])
    if res.success and _pose_error(pose, res.x) < 0.001:
        print(f"success:{res.success}")    
        return res.x
    print("fail")
    return None

In [2614]:
grasp_pose = grasp_reconst(xsol)
posevecs = grasp_to_posevecs(grasp_pose)

@jax.jit
def is_collision(q, grasp_pose=None):
    grasp_pose = None
    if grasp_pose is None:
        points = panda_model.get_surface_points_fn(q)
    else:
        robot_points = panda_model.get_surface_points_fn(q)
        ee_pose = SE3(panda_model.fk_fn(q)[-1])
        obj_pose = ee_pose @ grasp_pose.inverse()
        grasped_obj_points = jax.vmap(obj_pose.apply)(obj_points)
        points = jnp.vstack([robot_points, grasped_obj_points])
    return env.penetration_sum(points) != 0.

# check key poses
ee_pick = SE3(posevecs[0])
ee_place = SE3(posevecs[1])
key_poses = [ee_pick, ee_place]
for pose in key_poses:
    q = inverse_kinematics(pose)
    if is_collision(q):
        print("There is collision in IK")
    

success:True
success:True


In [2284]:
#grasp_offset = SE3.from_translation(jnp.array([0,0,depth]))
pre_pose = SE3.from_translation(jnp.array([0,0,-0.1]))
up_pose = SE3.from_translation(jnp.array([0,0,0.1]))
grasp_pose = grasp_reconst(xsol)
posevecs = grasp_to_posevecs(grasp_pose)

ee_pick = SE3(posevecs[0])
ee_place = SE3(posevecs[1])
q_pick = inverse_kinematics(ee_pick)
q_place = inverse_kinematics(ee_place)

ee0_pre = ee_pick @ pre_pose
ee0_post = up_pose @ ee_pick
q_pick_pre = inverse_kinematics(ee0_pre, q_pick)
q_pick_post = inverse_kinematics(ee0_post, q_pick)

ee1_pre = up_pose @ ee_place
ee1_post = ee_place @ pre_pose
q_place_pre = inverse_kinematics(ee1_pre, q_place)
q_place_post = inverse_kinematics(ee1_post, q_place)

qs = [panda.neutral, q_pick_pre, q_pick, q_pick_post, q_place_pre, q_place, q_place_post, panda.neutral]
# ee_poses = [ee0_pre, ee_pick, ee0_post, ee1_pre, ee_place, ee1_post]

# qs = []
# for ee in ee_poses:
#     qs += [inverse_kinematics(ee)]
# qs.insert(0, panda.neutral)
# qs += [panda.neutral]

success:True
success:True
success:True
success:True
success:True
success:True


In [2591]:
from copy import deepcopy
class Config:
    def __init__(self, q, parent=None):
        self.q = q
        self.parent = parent
    def __repr__(self):
        return self.q.__repr__()

def nearest_node(node_rand, tree):
    node_near = tree[0]
    for node in tree:
        if np.linalg.norm(node.q - node_rand.q) < np.linalg.norm(node_near.q - node_rand.q):
            node_near = node
    return node_near

def backtrack(last_node):
    path = [last_node.q]
    while True:
        if last_node.parent is None:
            break
        last_node = last_node.parent
        path.append(last_node.q)
    return path[::-1]

def get_direct_path(q1, q2, grasp_pose=None, partial=False):
    q_diff = q2-q1
    num_interp = np.ceil(np.linalg.norm(q_diff) / 0.1).astype(int)
    ratio = np.linspace(0,1, num_interp, endpoint=True)
    path = [q1 + q_diff*r for r in ratio][1:]
    i = 0
    for q in path:
        if is_collision(q, grasp_pose):
            if not partial: return None
            break
        i += 1
    if i == 0:  return None
    return path[:i+1]

def rrt(q_start, q_goal, grasp_pose=None):
    start = Config(q_start)
    goal = Config(q_goal)
    tree1 = [start]
    tree2 = [goal]
    
    direct_path = get_direct_path(q_start, q_goal, grasp_pose)
    if direct_path is not None:
        print("end: direct path")
        return direct_path
    
    tree_a, tree_b = tree1, tree2
    is_fwd = True

    for _ in range(200):
        node_rand = Config(panda.get_random_config())
        node_near_a = nearest_node(node_rand, tree_a)
        path = get_direct_path(node_near_a.q, node_rand.q, grasp_pose=grasp_pose, partial=True)
        if path is None: continue

        parent = node_near_a
        for q in path:
            node = Config(q, parent)
            tree_a.append(node)
            parent = node
        
        node_a = deepcopy(node)
        node_near_b = nearest_node(node_rand, tree_b)
        path = get_direct_path(node_near_b.q, node_a.q, grasp_pose=grasp_pose, partial=True)
        if path is None: continue

        parent = node_near_b
        for q in path:
            node = Config(q, parent)
            tree_b.append(node)
            parent = node

        if np.allclose(path[-1], node_a.q): #connected
            node_fwd, node_bwd = node_a, node
            if not is_fwd:  node_fwd, node_bwd = node_bwd, node_fwd
            path_fwd = backtrack(node_fwd)
            path_bwd = backtrack(node_bwd)[::-1]
            path = [q for q in (*path_fwd, *path_bwd)] 
            return path
        tree_a, tree_b = tree_b, tree_a
        is_fwd = not is_fwd

    return None


def smooth_path(path, grasp_pose=None):
    smoothed_path = path
    for _ in range(50):
        i = np.random.randint(0, len(smoothed_path) -1)
        j = np.random.randint(0, len(smoothed_path) -1)
        if abs(i - j) <= 1:
            continue
        if j < i:
            i, j = j, i
        shortcut = get_direct_path(smoothed_path[i], smoothed_path[j], partial=True)
        if shortcut is None: continue
        if (len(shortcut) < (j - i)) and all(not is_collision(q, grasp_pose) for q in shortcut):
            smoothed_path = smoothed_path[:i + 1] + shortcut + smoothed_path[j + 1:]
    print("smoothed")
    return smoothed_path

In [2615]:
import time
# move-pick-move-place-move

class Move:
    def __init__(self, q1, q2, grasp=None):
        self.q1 = q1
        self.q2 = q2
        self.grasp_pose = None
        self.path = self.get_path(q1, q2)
        if grasp is not None:
            self.grasp_pose = grasp_reconst(grasp)
        self.path = smooth_path(self.path, None)
    
    def get_path(self, q1, q2, num=20):
        return rrt(q1, q2, None)
    
    def show(self):
        for q in self.path:
            panda.set_joint_angles(q)
            ee_pose = SE3(panda_model.fk_fn(q)[-1])
            if self.grasp_pose is not None:
                obj.set_pose(ee_pose @ self.grasp_pose.inverse())
            time.sleep(0.05)

class Pick:
    def __init__(self, q1, q2, q3, grasp):
        self.q1 = q1
        self.q2 = q2
        self.q3 = q3
        self.path = self.get_path(q1, q2)
        self.path = smooth_path(self.path)
        self.grasp_pose = grasp_reconst(grasp)
    
    def get_path(self, q1, q2, num=20):
        return rrt(q1, q2)

    def show(self):
        for q in self.path:
            panda.set_joint_angles(q)
            time.sleep(0.05)
        for q in self.get_path(self.q2, self.q3):
            panda.set_joint_angles(q)
            ee_pose = SE3(panda_model.fk_fn(q)[-1])
            obj.set_pose(ee_pose @ self.grasp_pose.inverse())
            time.sleep(0.05)

class Place:
    def __init__(self, q1, q2, q3, grasp):
        self.q1 = q1
        self.q2 = q2
        self.q3 = q3
        self.path = self.get_path(q1, q2)
        self.path = smooth_path(self.path)
        self.grasp_pose = grasp_reconst(grasp)
    
    def get_path(self, q1, q2, num=20):
        return rrt(q1, q2)

    def show(self):
        for q in self.path:
            panda.set_joint_angles(q)
            ee_pose = SE3(panda_model.fk_fn(q)[-1])
            obj.set_pose(ee_pose @ self.grasp_pose.inverse())
            time.sleep(0.05)
        for q in self.get_path(self.q2, self.q3):
            panda.set_joint_angles(q)
            time.sleep(0.05)



In [1564]:
obj = Mesh(world.vis, "obj", "./sdf_world/assets/object/mesh.obj",
                alpha=0.8, color="blue")
obj.set_translate([0.4, -0.3, h/2])

In [2616]:
move1 = Move(qs[0], qs[1])
pick = Pick(qs[1], qs[2], qs[3], xsol)
move2 = Move(qs[3], qs[4], xsol)
place = Place(qs[4], qs[5], qs[6], xsol)
move3 = Move(qs[6], qs[7])

smoothed
end: direct path
smoothed
smoothed
end: direct path
smoothed
smoothed


In [2618]:
move1.show()
pick.show()
move2.show()
place.show()
move3.show()

end: direct path
end: direct path


In [2459]:
q_start = panda.neutral
q_goal = qs[3]

path = rrt(q_start, q_goal)

end


In [53]:
dt = 1/10

In [773]:
dim = 7
n = 10

to_mat = lambda vec:vec.reshape(-1, dim)
to_vec = lambda vec:vec.flatten()

def vector_integrate(x_init, xdots):
    dt_vec = jnp.full(n, dt)    
    lower_tri = jnp.tril(jnp.ones((n,n)))
    lower_tri = jnp.kron(lower_tri @ jnp.diag(dt_vec), jnp.eye(7))
    xs = to_vec(x_init + to_mat(lower_tri @ xdots))
    return xs
def rollout(state, u):
    q, qdot = state
    qdots = vector_integrate(qdot, u)
    qs = vector_integrate(q, qdots)
    return qs

In [975]:
def goal_pose_reaching_cost(q, target_pose):
    pose_curr = SE3(panda_model.fk_fn(q)[-1])
    pos_error = pose_curr.translation() - target_pose.translation()
    pos_error = jnp.sum(pos_error **2)
    orn_error = (target_pose.inverse()@pose_curr).rotation().log() /3
    orn_error = jnp.sum(orn_error**2)
    return pos_error + orn_error

In [977]:
#initialize
state_curr = panda.neutral, jnp.zeros(dim)
u = np.zeros(dim*n)

In [89]:
frame = Frame(world.vis, "frame")

In [781]:
qtn = np.random.normal(size=4)
qtn = qtn/jnp.linalg.norm(qtn)
goal_pose = SE3.from_rotation_and_translation(
    SO3(qtn), jnp.array([0.4, 0, 0.4]))
frame.set_pose(goal_pose)

In [978]:
def cost(state, u):
    qs = rollout(state, u)
    return jax.vmap(goal_pose_reaching_cost, in_axes=(0, None))(
        to_mat(qs), goal_pose
    ).sum()

In [1034]:
draw_ee_traj(u)

In [1199]:
from sdf_world.nlp import NLP

In [1206]:
mpc = NLP()
for i in range(n):
    mpc.add_var(f"u{i}", dim, panda.lb, panda.ub)
    mpc.add_var(f"q{i}", dim, np.ones(7)*-1, np.ones(7))

In [1461]:
robot_dim = 7
num_steps = 30

to_mat = lambda vec:vec.reshape(-1, robot_dim)
to_vec = lambda vec:vec.flatten()

get_qs = lambda x: x[:robot_dim*num_steps]
get_qdots = lambda x: x[robot_dim*num_steps:-robot_dim*num_steps]
get_us = lambda x: x[-robot_dim*num_steps:]
qs = jnp.tile(panda.neutral, num_steps)
qdots = jnp.zeros(robot_dim*num_steps)
us = jnp.zeros(robot_dim*num_steps)
x = jnp.hstack([qs, qdots, us])
u_lb, u_ub = np.ones(robot_dim)*-1, np.ones(robot_dim)
qdot_lb, qdot_ub = np.ones(robot_dim)*-1, np.ones(robot_dim)

dim = x.shape[0]
lb = np.hstack([
    np.tile(panda.lb, num_steps), 
    np.tile(qdot_lb, num_steps),
    np.tile(u_lb, num_steps)
])
ub = np.hstack([
    np.tile(panda.ub, num_steps), 
    np.tile(qdot_ub, num_steps),
    np.tile(u_ub, num_steps)
])

In [1462]:
#dynamic constraints
def constr_init_goal(x):
    qs = to_mat(get_qs(x))
    q_init, q_goal = qs[0], qs[-1]
    return jnp.hstack([q_init-panda.neutral, q_goal-panda.neutral])

def constr_dynamics(x):
    qs = to_mat(get_qs(x))
    qdots = to_mat(get_qdots(x))
    us = to_mat(get_us(x))

    err_vel_dyn = qs[:-1] + qdots[:-1]*dt  - qs[1:]
    err_acc_dyn = qdots[:-1] + us[:-1]*dt  - qdots[1:]
    return jnp.hstack([err_vel_dyn.flatten(), err_acc_dyn.flatten()])

def constr_fn(x):
    return jnp.hstack([
        constr_init_goal(x),
        constr_dynamics(x)
    ])

_jac_constr_fn = jax.jacrev(constr_fn)
row, col = np.nonzero(_jac_constr_fn(np.random.random(dim)))
jac_constr_fn = lambda x: _jac_constr_fn(x)[row, col]
jac_constr_fn = jax.jit(jac_constr_fn).lower(x).compile()
jac_struct_fn = lambda : (row, col)
constr_fn = jax.jit(constr_fn).lower(x).compile()
const_dim = constr_fn(x).shape[0]

In [1516]:
get_tcp = lambda q: panda_model.fk_fn(q)[-1][-3:]
def objective(x):
    qs = to_mat(get_qs(x))
    tcps = jax.vmap(get_tcp)(qs)
    target_point = np.array([0.4, 0.4, 0.4])
    return jnp.sum((tcps - target_point).flatten()**2)
grad_fn = jax.jit(jax.grad(objective)).lower(x).compile()
obj_fn = jax.jit(objective).lower(x).compile()

In [1517]:
class MPCProb:
    pass    
mpc = MPCProb()
setattr(mpc, "objective", obj_fn)
setattr(mpc, "gradient", grad_fn)
setattr(mpc, "constraints", constr_fn)
setattr(mpc, "jacobian", jac_constr_fn)
setattr(mpc, "jacobianstructure", jac_struct_fn)

In [1518]:
import cyipopt
ipopt = cyipopt.Problem(
    n=dim, m=const_dim,
    problem_obj=mpc,
    lb=lb, ub=ub,
    cl=jnp.zeros(const_dim), 
    cu=jnp.zeros(const_dim)
)
ipopt.add_option("acceptable_tol", 0.01)
ipopt.add_option("acceptable_constr_viol_tol", 0.01)
ipopt.add_option("acceptable_iter", 2)

In [1522]:
mpc.gradient(x)

Array([-4.90109503e-01, -9.50984508e-02, -4.90109473e-01,  2.03994036e-01,
       -1.41606554e-01,  1.20430313e-01,  3.72529030e-09, -3.81158859e-01,
       -5.77685833e-02, -3.72178853e-01,  1.34905607e-01, -1.19563460e-01,
        7.55352974e-02, -1.72239112e-09, -2.72918075e-01, -2.59929933e-02,
       -2.62308151e-01,  7.65750781e-02, -9.27229822e-02,  4.04165611e-02,
        9.14376130e-10, -1.76066607e-01, -3.44259362e-03, -1.67656764e-01,
        3.30046341e-02, -6.43205270e-02,  1.63540244e-02,  5.86894089e-10,
       -9.62055698e-02,  8.71504098e-03, -9.13946256e-02,  5.52916434e-03,
       -3.71367037e-02,  2.78617488e-03,  1.00207931e-09, -3.44693176e-02,
        1.10447705e-02, -3.31875756e-02, -7.66679458e-03, -1.36465877e-02,
       -2.95832008e-03, -3.30039995e-10,  9.82091483e-03,  5.49811590e-03,
        8.39256216e-03, -9.67822690e-03,  4.39268490e-03, -3.67637910e-03,
        2.66421551e-10,  3.67679670e-02,  1.17470848e-03,  3.29275802e-02,
       -1.01040518e-02,  

In [1520]:
%timeit mpc.gradient(x)

238 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [1476]:
x, info = ipopt.solve(x)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     1232
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      630
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      630
                     variables with only upper bounds:        0
Total number of equality constraints.................:      420
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.1086924e-01 9.28e-03 1.04e-02   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [1477]:
i = 0

In [1507]:
q = to_mat(get_qs(x))[i]
tcp = panda_model.fk_fn(q)[-1][-3:]
panda.set_joint_angles(to_mat(get_qs(x))[i])

print(tcp)
i += 1

[6.1263680e-01 4.6566129e-10 5.5448973e-01]


In [1469]:
%time
mpc.jacobian(x)

CPU times: user 13 µs, sys: 2 µs, total: 15 µs
Wall time: 32.2 µs


Array([ 1. ,  1. ,  1. , ...,  1. , -1. ,  0.1], dtype=float32)

In [ ]:
mpc.add_var("x", 105, )

In [1244]:
x.shape

(105,)

In [1239]:
err

Array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [1221]:
qs[0]

Array([ 0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ],      dtype=float32)

In [1217]:
get_qs(x)

Array([ 0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ],      dtype=float32)

In [1216]:
x

Array([ 0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    , -1.5708,  0.    ,  1.8675,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],      dtype=float32)

In [ ]:

mpc.add_var()

In [1030]:
u_grad = jax.grad(cost, argnums=1)(state_curr, u)
u_hess = jax.hessian(cost, argnums=1)(state_curr, u)

In [1179]:
u_grad = jax.grad(cost, argnums=1)(state_curr, u)
u = u - u_grad * 1.
draw_ee_traj(u)

print(cost(state_curr, u))

0.49036166


In [1180]:
i = 0

In [1197]:
qs = rollout(state_curr, u)
panda.set_joint_angles(to_mat(qs)[i])
i += 1

In [ ]:
q

In [755]:
world.vis["line"].delete()

In [759]:
import meshcat.geometry as g
get_ee_pos = lambda q: panda_model.fk_fn(q)[-1][-3:]
def draw_ee_traj(u):
    qs = rollout(state_curr, u)    
    vertices = jax.vmap(get_ee_pos)(to_mat(qs))
    world.vis["line"].set_object(
        g.Line(
            g.PointsGeometry(vertices.T),
            g.MeshLambertMaterial(color=0xff0000)
        )
    )

In [128]:
qs = rollout(state_curr, unew)

Array([ 2.1451635e-03,  9.0127572e-04,  2.1451635e-03, -1.5716413e+00,
       -6.1933900e-04,  1.8666126e+00, -2.2325052e-03,  6.0454607e-03,
        2.5399588e-03,  6.0454607e-03, -1.5731711e+00, -1.7454098e-03,
        1.8649993e+00, -6.2916055e-03,  1.1349866e-02,  4.7685681e-03,
        1.1349866e-02, -1.5752516e+00, -3.2768664e-03,  1.8628051e+00,
       -1.1811983e-02,  1.7746355e-02,  7.4560083e-03,  1.7746352e-02,
       -1.5777605e+00, -5.1236227e-03,  1.8601592e+00, -1.8468907e-02,
        2.4961904e-02,  1.0487571e-02,  2.4961902e-02, -1.5805906e+00,
       -7.2068535e-03,  1.8571745e+00, -2.5978243e-02,  3.2762501e-02,
        1.3764938e-02,  3.2762498e-02, -1.5836501e+00, -9.4589954e-03,
        1.8539478e+00, -3.4096442e-02,  4.0953126e-02,  1.7206172e-02,
        4.0953122e-02, -1.5868627e+00, -1.1823744e-02,  1.8505597e+00,
       -4.2620555e-02,  4.9377769e-02,  2.0745728e-02,  4.9377762e-02,
       -1.5901670e+00, -1.4256057e-02,  1.8470749e+00, -5.1388212e-02,
      

In [133]:
panda.set_joint_angles(to_mat(qs)[9])

In [120]:
to_mat(qs)

Array([[-3.2211264e-04,  7.6030346e-04, -3.2211264e-04, -1.5715439e+00,
        -2.2423945e-04,  1.8667434e+00,  2.4247805e-04],
       [-8.5896702e-04,  2.0274757e-03, -8.5896702e-04, -1.5727838e+00,
        -5.9797184e-04,  1.8654826e+00,  6.4660812e-04],
       [-1.5246664e-03,  3.5987697e-03, -1.5246664e-03, -1.5743214e+00,
        -1.0614000e-03,  1.8639191e+00,  1.1477295e-03],
       [-2.2547883e-03,  5.3221239e-03, -2.2547883e-03, -1.5760077e+00,
        -1.5696762e-03,  1.8622044e+00,  1.6973463e-03],
       [-3.0063845e-03,  7.0961653e-03, -3.0063845e-03, -1.5777436e+00,
        -2.0929016e-03,  1.8604393e+00,  2.2631285e-03]], dtype=float32)

In [88]:
jax.vmap(goal_pose_reaching_cost, in_axes=(0, None))(to_mat(qs), SE3.identity())

Array([ 9.91998 , 10.063185, 10.306283, 10.559819, 10.092124], dtype=float32)

In [76]:
q = to_mat(qs)[0]

In [81]:
pose_curr = SE3(panda_model.fk_fn(q)[-1])

In [82]:
pose_curr

SE3(wxyz=[0.05169 0.91142 0.38204 0.14382], xyz=[0.6174  0.01408 0.55513])

In [70]:
qs = rollout(state_curr, u)

In [71]:
qs

Array([ 0.01      ,  0.01      ,  0.01      , -1.5608    ,  0.01      ,
        1.8774999 ,  0.01      ,  0.03      ,  0.03      ,  0.03      ,
       -1.5408    ,  0.03      ,  1.8974999 ,  0.03      ,  0.06      ,
        0.06      ,  0.06      , -1.5107999 ,  0.06      ,  1.9275    ,
        0.06      ,  0.10000001,  0.10000001,  0.10000001, -1.4707999 ,
        0.10000001,  1.9675    ,  0.10000001,  0.15      ,  0.15      ,
        0.15      , -1.4208    ,  0.15      ,  2.0175    ,  0.15      ],      dtype=float32)

In [69]:
panda.set_joint_angles(to_mat(qs)[4])

In [19]:
panda

In [18]:
to_mat(qs)[2]

Array([ 5.9999998e-05,  5.9999998e-05,  5.9999998e-05, -1.5707400e+00,
        5.9999998e-05,  1.8675599e+00,  5.9999998e-05], dtype=float32)

In [2686]:
to_mat(qs)

Array([[ 1.00000007e-05,  1.00000007e-05,  1.00000007e-05,
        -1.57078993e+00,  1.00000007e-05,  1.86750996e+00,
         1.00000007e-05],
       [ 3.00000029e-05,  3.00000029e-05,  3.00000029e-05,
        -1.57076991e+00,  3.00000029e-05,  1.86752999e+00,
         3.00000029e-05],
       [ 5.99999985e-05,  5.99999985e-05,  5.99999985e-05,
        -1.57073998e+00,  5.99999985e-05,  1.86755991e+00,
         5.99999985e-05],
       [ 1.00000005e-04,  1.00000005e-04,  1.00000005e-04,
        -1.57069993e+00,  1.00000005e-04,  1.86759996e+00,
         1.00000005e-04],
       [ 1.50000007e-04,  1.50000007e-04,  1.50000007e-04,
        -1.57064998e+00,  1.50000007e-04,  1.86764991e+00,
         1.50000007e-04]], dtype=float32)

In [2650]:
lower_tri_q = jnp.kron(lower_tri, jnp.eye(7)).astype(int)

In [2652]:
lower_tri_q.shape

(35, 35)

In [2461]:
i=0

In [2514]:
panda.set_joint_angles(path[i])
i+= 1

IndexError: list index out of range

In [2516]:
panda.set_joint_angles(qs[3])

In [2382]:
tree = [Config(q_start)]
step_size = 0.2
for _ in range(1000):
    if np.random.random() < 0.3:
        node_rand = Config(q_goal)
    else:
        node_rand = Config(panda.get_random_config())
    node_near = nearest_node(node_rand, tree)
    diff = node_rand.q - node_near.q
    q_new = node_near.q + step_size*(diff) / np.linalg.norm(diff)
    if not is_collision(q_new):
        panda.set_joint_angles(q_new)
        node_new = Config(q_new, node_near)
        tree.append(node_new)
        if np.linalg.norm(q_goal - q_new) < step_size:
            print("end")
            #return node_new

end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end
end


In [2356]:
q_goal = 
near_node = nearest_node(Config(q_goal), tree)

In [2363]:
near_node = near_node.parent
panda.set_joint_angles(near_node.q)

AttributeError: 'NoneType' object has no attribute 'q'

In [ ]:
panda.get_random_config()

array([-2.27098848,  0.72912031,  1.32183182, -1.34565211,  2.03104704,
        0.90633754,  0.03920187])

In [ ]:
from sdf_world.nlp import NLP
mp = NLP()

In [ ]:
#q0, q1, q2, q3, q4
mp.add_var("qinit", 7, panda.neutral, panda.neutral)
mp.add_var("qmove", 7, panda.lb, panda.ub)
mp.add_var("qpick", 7, panda.lb, panda.ub)
mp.add_var("qmovehold", 7, panda.lb, panda.ub)
mp.add_var("qplace", 7, panda.lb, panda.ub)
mp.add_var("qhoming", 7, panda.lb, panda.ub)
mp.add_var("qhome", 7, panda.neutral, panda.neutral)

In [ ]:
q_pick

array([ 1.44554337,  1.82764066, -0.97383876, -2.44912557,  0.39810294,
        2.66130424,  2.64500663])

In [ ]:
is_collision(q_pick)

Array(True, dtype=bool)

In [ ]:
panda.set_joint_angles(q_pick)

In [ ]:
pick.show()

In [2367]:
move1.show()
pick.show()
move2.show()
place.show()
move3.show()

In [ ]:
mp = NLP()

In [ ]:
mp.add_var("q0", 7, lower=qs[0], upper=qs[0])
mp.add_var("qgoal", 7, lower=qs[-1], upper=qs[-1])


In [ ]:
qs_cont = []
for i in range(len(qs)-1):
    q1, q2 = qs[i], qs[i+1]
    qs_cont.append(jnp.linspace(q1, q2, 5))
qs_cont = jnp.vstack(qs_cont)
qs_path = jnp.hstack([
    jnp.vstack(qs_cont), 
    jnp.full((len(qs_cont),2), 0.04)])

In [ ]:
world.vis["obj_pc"].delete()

In [ ]:
obj_points = obj.mesh.sample(20)
def get_obj_pc(q, grasp):
    ee_pose = SE3(panda_model.fk_fn(q)[-1])
    grasp_pose = grasp_reconst(grasp)
    obj_pose = ee_pose @ grasp_pose.inverse()
    return jax.vmap(obj_pose.apply)(obj_points)
get_move_hold_pc = jax.vmap(get_obj_pc, in_axes=(0, None))

In [ ]:
def get_path_points(qs):
    return jnp.vstack(jax.vmap(panda.get_surface_points_fn)(qs))
def get_object_point(q, grasp_pose):
    ee_pose = SE3(panda_model.fk_fn(q)[-1])
    obj_pose = ee_pose @ grasp_pose.inverse()
    return jax.vmap(obj_pose.apply)(obj_points)
def get_hold_path_points(qs, grasp):
    grasp_pose = grasp_reconst(grasp)
    obj_points = jnp.vstack(jax.vmap(get_object_point, in_axes=(0, None))(qs, grasp_pose))
    path_points = get_path_points(qs)
    return jnp.vstack([obj_points, path_points])

In [ ]:
qs_movehold = jnp.linspace(qs[3], qs[4], 4)

In [ ]:
points = get_hold_path_points(qs_movehold, xsol)

In [ ]:
del pc_path

In [ ]:
pc_path = PointCloud(world.vis, "pc_path", points, 0.01, "blue")

In [ ]:
del pc_path

In [ ]:
obj.set_translate([0.4, -0.3, h/2])
move1.show()
pick.show()
move2.show()
place.show()
move3.show()

In [ ]:
i = 0

In [ ]:
panda.set_joint_angles(qs[i])
i+=1

In [ ]:
obj = Mesh(world.vis, "obj", "./sdf_world/assets/object/mesh.obj",
                alpha=0.8, color="blue")
obj.set_translate([0.4, -0.3, h/2])
obj_points = obj.mesh.sample(20)
obj.set_translate([0.4, -0.3, h/2])

In [ ]:
class Pick:
    def __init__(self):
        pass

In [ ]:
paths = []
for i in range(len(qs)-1):
    q1, q2 = qs[i], qs[i+1]
    path = np.linspace(q1, q2, 20)
    paths.append(path)
paths = np.vstack(paths)

In [ ]:

for q in paths:
    panda.set_joint_angles(q)
    time.sleep(0.05)

In [ ]:
posevecs = grasp_to_posevecs(g)
logit = get_grasp_logit(g)
penet = penetration(posevecs)
manips = manipulability(posevecs)
print(logit, penet, manips)

-14.847334 3.2866842e-06 [0.48995048 0.5860477 ]


In [ ]:
jax.grad(get_grasp_logit)(g)

Array([-1.2881178 ,  0.16389363,  0.04734417], dtype=float32)

In [ ]:
manipulability(posevecs)

Array([0.51384914, 0.58879846], dtype=float32)

In [ ]:
jax.grad(get_grasp_logit)(g)

Array([-1.2881178 ,  0.16389363,  0.04734417], dtype=float32)

In [ ]:
get_grasp_logit(g)

Array(-14.845617, dtype=float32)

In [ ]:
from sdf_world.nlp import NLP
prob = NLP()

In [ ]:
constraints(g)

Array([-1.8105585e+01,  7.8934552e-03,  5.7361335e-01,  6.2390339e-01],      dtype=float32)

In [ ]:
posevecs = grasp_to_posevecs(g)
penetration(posevecs)

Array(0.00789346, dtype=float32)

In [ ]:
points[:,-1].min()

Array(0.02936802, dtype=float32)

In [ ]:
posevecs = grasp_to_posevecs(g)
points = assign_hands_points_fn(posevecs)
pc = PointCloud(world.vis, "pc", points, 0.01, "red")

In [ ]:
world.vis["pc"].delete()

In [ ]:
penetration()

Array(0.00789346, dtype=float32)

In [ ]:
get_grasp_to_posevecs(g)

Array([[ 0.3635179 ,  0.4089301 , -0.35115376,  0.75981706,  0.36155713,
        -0.21275215,  0.06269744],
       [-0.03211129,  0.54620326, -0.78557503,  0.28896856,  0.36155713,
         0.34398812,  0.16934496]], dtype=float32)

In [ ]:
get_grasp_logit(g)

Array(-18.105585, dtype=float32)